<a href="https://colab.research.google.com/github/SpatialeLearning/ee-python-colab/blob/main/03_CART_Landsat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **CART Supervised Classification**

**Authenticate and Initialize Earth Engine**

In [2]:
#Import earth engine
import ee
 
# Trigger the authentication flow.
ee.Authenticate()
 
# Initialize the library.
ee.Initialize()

In [ ]:
# CART Supervised Classification with Landsat

#ROI
roi = ee.Geometry.Point([-122.26, 37.87])

# Load Landsat 8 data
image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
.filterDate('2017-01-01', '2017-12-31')\
.filterBounds(roi)\
.sort('CLOUD_COVER')\
.first()
          
visParamsTrue = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000, 'gamma': 1.4}

In [ ]:
# Import training data
training = ee.FeatureCollection('users/midekisa/Train_Cover_CA')

label = 'Class'
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B7']
input = image.select(bands)

# Overlay the points on the imagery to get training.
trainImage = input.sampleRegions(**{
  'collection': training,
  'properties': [label],
  'scale': 30
})

trainingData = trainImage.randomColumn()
trainSet = trainingData.filter(ee.Filter.lessThan('random', 0.8))
testSet = trainingData.filter(ee.Filter.greaterThanOrEquals('random', 0.8))


In [ ]:
# Classification Model
classifier = ee.Classifier.smileCart().train(trainSet, label, bands)

# Classify the image
classified = input.classify(classifier)
#print(classified.getInfo())

# Define a palette for the classification.
landcoverPalette = [
  '#0c2c84', #water (0)
  '#e31a1c', #urban (1)
  '#005a32', #forest (2)
  'FF8000', #crop (3)
  '969696', #barren (4)
]

In [ ]:

# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
my_map = folium.Map(location=[38, -124], zoom_start = 8)

# Add the layer to the map object.
my_map.add_ee_layer(image, visParamsTrue, "Landsat 2017")
my_map.add_ee_layer(classified, {'palette': landcoverPalette, 'min': 0, 'max':4 }, 'classification')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)